In [1]:
%time
%load_ext autotime
%load_ext autoreload
%autoreload 2


# if cannot import the modules, add the parent directory to system path might help

import os
import sys
parent_dir = os.path.abspath(os.getcwd()+'/..')+'/'
sys.path.append(parent_dir)


import os
from utils.path import dir_HugeFiles
from utils.preprocessing import load
from utils.words import make_corpus_0, get_wordcount_list
from utils.save import make_dir, save_pickle, load_pickle, auto_save_csv, print_time
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import spacy
import random
import re

#dir_save = os.path.normpath(dir_HugeFiles+'dph/dic_20190607.pickle')
#dic = load(dir_save)
random_seed = 2019

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.91 µs


In [26]:
dic = load(dir_save = '../big_data/dic_regional_20190925.pickle')

ls = [i for i,v in dic.items() if len(v['ingredients'])>1]
print('drop %d recipes with less than 2 ingredients' %(len(dic)-len(ls)))
ls = [i for i in ls if len(dic[i]['directions'])>1]
print('furthur drop %d recipes with less than 2 instructions' %(len(dic)-len(ls)))
desc = [i for i in ls if len(dic[i]['description'])<1]
print('drop %d recipes with no description' %(len(desc)))
print('now we are using recipe54k %d' % len(ls))

exist
drop 46 recipes with less than 2 ingredients
furthur drop 1026 recipes with less than 2 instructions
drop 0 recipes with no description
now we are using recipe54k 54076
time: 9.83 s


In [27]:
styles = {'chinese':[],'general western':[],
          'indian':[], 'italian':[],'latin american':[]}

for i, v in dic.items():
    if 'regional_tag3' in v.keys():
        # filter out the ' recipe'
        recipe = set([t[:-8] if t[-8:]==' recipes' else t for t in v['tags']])
        if 'main dish' in recipe and 'desserts' in recipe:
            dic[i]['meal_type'] = 'dual_type'
        elif 'main dish' in recipe:
            dic[i]['meal_type'] = 'main'
        elif 'main dishes' in recipe:
            dic[i]['meal_type'] = 'main'
        elif 'desserts' in recipe:
            dic[i]['meal_type'] = 'desserts'
        else:
            dic[i]['meal_type'] = 'others'
            
        if dic[i]['meal_type'] == 'main':
            if v['regional_predict'] in styles.keys():
                styles[v['regional_predict']].append(i)

time: 480 ms


In [25]:
len(styles['chinese'])

0

time: 22.5 ms


In [28]:
def reverse(text):
    # replace things in brace
    text = re.sub(r'\([^)]*\)', '', text)

    # remove space before punct
    text = re.sub(r'\s([?.!,"](?:\s|$))', r'\1', text)
    
    # remove consecutive spaces
    text = re.sub(' +',' ',text).strip()
    return text

def txt(v, fields, mode = 'train'):
    '''
    fields: an order list, the last is the field to predict
    mode: test/train, return string X, y or X+y
    '''
    to_write = ''
    for field in fields:
        if field == 'style':
            to_write += ' <start-style>'+v['regional_predict']+' <end-style>'
        if field == 'ingredients':
            ingredients = [reverse(sent) for sent in v['ny_ingredients']['ny'] if type(sent)==str]
            to_write += ' <start-ingredients>'+'$'.join(ingredients)+'$ <end-ingredients>'
        if field == 'directions':
            directions = [reverse(sent) for sent in v['directions']]
            to_write += ' <start-directions>'+' '.join(directions)+' <end-directions>'
                                                     
    if mode == 'train':
        return to_write
                                                     
    elif mode == 'test':
        field_to_predict = '<start-%s>'%fields[-1]
        to_X, to_y = to_write.split(field_to_predict)
        return to_X + field_to_predict, to_y

class to_gpt2:
    def __init__(self, dic, styles):
        self.random = random
        self.random.seed(2019)
        ls = sum([self.random.sample(ls_style,400) for i, ls_style in styles.items()], [])
        classes = [dic[i]['regional_predict'] for i in ls]
        ls_train, self.ls_test, class_train , __ = train_test_split(ls, classes, 
                                                         test_size = 0.2, 
                                                         random_state = random_seed, 
                                                         shuffle = True, 
                                                         stratify = classes)
        self.ls_train, self.ls_val, _, __ = train_test_split(ls_train, 
                                                             class_train, 
                                                             test_size = 0.25, 
                                                             random_state = random_seed ,
                                                             shuffle = True,
                                                             stratify = class_train)
        self.dic = dic
    
    def train(self, ls, filename, overwrite = False, is_val = False):
        to_write = ''
        for i, v in self.dic.items():
            if i in ls: 
                self.save(filename+'%d'%(i)+'d.txt', txt(v, ['style','ingredients','directions']), overwrite)
                
    def test(self, ls, filename, overwrite = False, is_val = False):
        to_write = ''
        for i, v in self.dic.items():
            if i in ls:
                self.save(filename+'X/%d'%(i)+'d.txt', txt(v, ['style','ingredients','directions'], mode = 'test')[0], overwrite)                
                self.save(filename+'y/%d'%(i)+'d.txt', txt(v, ['style','ingredients','directions'], mode = 'test')[1], overwrite)
                self.save(filename+'Xy/%d'%(i)+'d.txt', txt(v, ['style','ingredients','directions']),overwrite)
                
    def fast_export(self, filename, overwrite = False):
        self.train(self.ls_train, filename+'train/', overwrite = overwrite)
        self.train(self.ls_val, filename+'val/',overwrite = overwrite)
        self.test(self.ls_test, filename+'test/', overwrite = overwrite)
    def save(self, filename, to_write, overwrite = False):
        make_dir(filename)
        if os.path.isfile(filename) == True and overwrite == False:
            print('already exists'+filename)
        else:    
            with open(filename,'w') as f:
                f.write('%s' % to_write)

time: 48.8 ms


In [48]:
model = to_gpt2(dic, styles)

time: 28.2 ms


In [49]:
model.fast_export('../../to_gpt2/recipe54k_0925/',overwrite = True)

make dir
make dir
make dir
make dir
make dir
time: 4.55 s


### delexicalized the sentences

In [6]:
df = pd.read_csv('../big_data/food_taxonomy.txt',delimiter='\t', header = None)

def cleaning(df):
    '''
    eliminate the row if it contains the following non-ingredient words
    '''
    print('origin: length of data: %d' % len(df))
    eliminate = ['Snack brand', 'Preparation', 'Fast food', 'Dietary Supplement', 'Dessert']
    for i in range(2):
        df  = df[df.apply(lambda x: x[i] not in eliminate, axis = 1)]
    # uncased
    for i in range(3):
        df[i]  = df.apply(lambda x: x[i].lower(), axis = 1)
    print('drop some fields: length of data: %d' % len(df))    
    
    # drop the duplicates
    df = df[~df[2].duplicated()]
    df = df.reset_index(drop = True)
    print('drop duplicates: length of data: %d' % len(df))
        
    return df

df = cleaning(df)
additional_ingr=\
[
['Condiment', 'Sweet', 'sugar'],
['Condiment', 'Sweet', 'applesauce'],
['Flour', 'Flour','flour'],
['Baking powder','Baking powder','baking powder'],
['Water','Water','water'],
['Water','Water','iced'],
['Water','Water','ice'],
['Herb and spice','spices','jalapeno'],
['Condiment','Condiment', 'oil'],
['Beverage','Fruit juice', 'juice'],
['Staple','Maize','enchilada'],
['Condiment','Condiment', 'dressing mix'],
['Condiment','Condiment', 'cheese mix'],
['Baking powder','Baking powder','baking soda'],
['Condiment','Condiment', 'dry mix'],
['Condiment','Condiment','chocolate'],
['Vegetable','Bulb and stem vegetables','tapioca'],
['Flour', 'Flour','xanthan gum'],
['Flour', 'Flour','starch'],
['Egg and dairy', 'Dairy product','buttermilk'],
['Condiment','Condiment','chili'],
['Condiment','Condiment','chile'],
['Condiment','Condiment','chilis'],
['Condiment','Condiment','chiles'],
['Condiment','Condiment','chilies'],
['Flour', 'Flour','corn muffin mix'],
['Beverage','Chocolate','chocolate mix'],
['Meat','dumpling','dumpling'],
['Meat','dumpling','wonton'],
['Staple','Wheat','pizza dough'],
['Staple','Wheat','dough'],
['Condiment','Condiment', 'pizza sauce'],
['Flour', 'Flour','yeast'],
['Condiment','Sweet','cocoa'],
['Staple','Maize','chip'],
['Egg and dairy','Dairy product','ricotta'],
['Condiment','Condiment','seasoning'],
['Beverage','Alcohol','sherry'],
['Staple','Rice','grain rice'], 
['Staple','Wheat','shell'],
['Meat','Beef','fillet'],
['Staple','Maize','cornmeal'],
['Condiment','Condiment','seed oil'],
['Nut and seed','Other','seed'],
['Condiment', 'Sweet', 'sugar blend'],
['Soup','Soup','broth'],
['Soup','Soup','stock'],
['Condiment', 'Sweet', 'marshmallow'],
['Condiment', 'Dry Condiment', 'dried vegetable flakes'],
['Condiment', 'Dry Condiment', 'dried celery flakes'],
['Flour', 'Flour','cornstarch'],
['Staple','Wheat','double crust'],
['Staple','Wheat','crust'],
['Staple','Wheat','pastry crust'],
['Egg and dairy','Dairy product','gorgonzola'],
['Beverage','juice','drink mix'],
['Egg and dairy','Egg','Egg whites'],
['Baking powder','Baking powder','baking mix'],
['Staple','Rice','brown rice'],
['Condiment','Condiment','five spice'],
['Meat','Beef','tenderloin'],
['Meat','Pork','prosciutto'],
['Condiment', 'Sweet', 'whipped topping'],
['Condiment', 'Sweet', 'topping'],
['Beverage','Alcohol','cider'],
['Meat','Shellfish','crabmeat'],
['Condiment', 'Sweet', 'candy'],
['Condiment', 'Sweet', 'caramel'],
['Condiment', 'Sweet', 'molasses'],
['Vegetable','Podded vegetables','cannellini'],
['Vegetable','Fruits','fruit'],
['Staple','Wheat','saltine'],
['Condiment','Condiment', 'habanero'],
['Beverage','Juice','jell o'],
['Beverage','Juice','jelly'],
['Beverage','Soft drink','carbonated beverage'],
['Egg and dairy','Dairy product','gruyere'],
['Vegetable','Leafy and Salad','beet'],
['Water','Water','icing'],
['Egg and dairy','Dairy product','parmigiano'],
['Beverage','Alcohol','liqueur'],
['Condiment','Condiment', 'lard'],
['Staple','Wheat','crumb'],
['Herb and spice','Herb','peppermint'],
['Beverage','Alcohol','marsala'],
['Side dish','Potatoes','hash brown'],
['Meat','Beef','steak'],
['Condiment','Condiment','gelatin'],
['Meat','Beef','chuck'],
['Egg and dairy','Dairy product','colby'],
['Condiment', 'Sweet', 'jam'],
['Condiment', 'Sweet', 'cool whip'],
['Condiment', 'Sweet', 'stevia'],
['Staple','Wheat','bran'],
['Condiment','Condiment','pimento'],
['Condiment','Condiment','food coloring'],
['Meat','Meat','rib'],
['Condiment','Condiment','shortening'],
['Vegetable','Fruits','sweet pickles'],
['Condiment', 'Sweet', 'white confectioner'],
['Condiment', 'Sweet', 'confectioner'],  
['Vegetable','Root and tuberous vegetabless','rhubarb'],
['Condiment', 'Condiment', 'cooking spray']
]

def add_rows(df, additional_ingr):
    add = pd.DataFrame(additional_ingr)
    print('before processing: length of data: %d' % len(df))
    df = pd.concat([df,add]).reset_index(drop =True)

    # uncased
    for i in range(3):
        df[i]  = df.apply(lambda x: x[i].lower().strip(), axis = 1)
    print('add some rows: length of data: %d' % len(df))    
    
    # drop the duplicates
    df = df[~df[2].duplicated()]
    df = df.reset_index(drop = True)
    print('drop duplicates: length of data: %d' % len(df))
    return df

df = add_rows(df, additional_ingr)

origin: length of data: 4294
drop some fields: length of data: 2682
drop duplicates: length of data: 2582
before processing: length of data: 2582
add some rows: length of data: 2685
drop duplicates: length of data: 2684
time: 1.64 s


In [43]:
import random
import spacy
class delexicalization:
    def __init__(self, df):
        ### 
        self.spacy = spacy.load('en_core_web_lg')  # 10 sec
        df[3] = df[2].apply(lambda x: self.lemma_ingr(x)) # 30 sec
        self.level3 = df.groupby([3])[1].apply(list).index.tolist()
        self.gb = df.groupby([3])[1].apply(list)
        self.level3_set = [set(ingr.split(' ')) for ingr in self.level3]
        self.random = random
        self.random.seed(2019)
        
    def lemma_ingr(self, ingr):
        '''
        Args: ingr: a str
        Return: str: the first exact match or no change
        '''
        assert type(ingr) == str
        phrases_to_sentences = 'Mix the %s and water.'%(ingr)
        doc = self.spacy(phrases_to_sentences)
        exact_match, root_match = [],[]
        for chunk in doc.noun_chunks:
            if chunk.text != 'water':
                root_lemma = [token.lemma_ for token in doc if token.text == chunk.root.text][0]
                exact_match.append(chunk.lemma_.replace('the ',''))
                root_match.append(root_lemma)
        if exact_match:
            return exact_match[0]
        else:
            return ingr
        
    def delexicalized(self, exact_ingr):
        '''
        input: dic[9]['ny_ingredients']['exact'][0]
        e.g. 'onion'
        '''
        matched = []
        for ingr, ingr_set in zip(self.level3, self.level3_set):
            if len(ingr_set) == len(ingr_set & set(exact_ingr.split(' '))):
                matched += self.gb[ingr]
        if matched:
            return '%s#%s' % (self.random.sample(matched, 1)[0], exact_ingr)
        
    def ingr(self, exact_ingr):
        matched = self.delexicalized(exact_ingr)
        if matched:
            return matched
        else:
            return exact_ingr
        
    def sent(self, sent):
        '''
        For example
        sent = 'Mix the porks, chicken, white sugar with the grain rice in a pot.'
        delexicalized_sent(sent) = 
        'Mix the pork#pork, poultry#chicken, sweet#white sugar with the rice#grain rice in a pot.'
        '''
        output = sent
        doc = self.spacy(sent)
        for chunk in doc.noun_chunks:
            food_matching = self.delexicalized(chunk.lemma_.replace('the ',''))
            if food_matching:
                output = output.replace(chunk.text.replace('the ',''), food_matching)
        return output
# takes 1 min
food_taxonomy = delexicalization(df)

time: 45.7 s


In [45]:
def txt(v, fields, mode = 'train'):
    '''
    fields: an order list, the last is the field to predict
    mode: test/train, return string X, y or X+y
    '''
    to_write = ''
    for field in fields:
        if field == 'style':
            to_write += ' <start-style>'+v['regional_predict']+' <end-style>'
        if field == 'ingredients':
            ingredients = [food_taxonomy.ingr(reverse(sent)) \
                           for sent in v['ny_ingredients']['exact'] if type(sent)==str]
            
            to_write += ' <start-ingredients>'+'$'.join(ingredients)+'$ <end-ingredients>'
        if field == 'directions':
            directions = [food_taxonomy.sent(reverse(sent)) for sent in v['directions']]
            to_write += ' <start-directions>'+' '.join(directions)+' <end-directions>'
                                                     
    if mode == 'train':
        return to_write
                                                     
    elif mode == 'test':
        field_to_predict = '<start-%s>'%fields[-1]
        to_X, to_y = to_write.split(field_to_predict)
        return to_X + field_to_predict, to_y

time: 33.4 ms


In [46]:
model = to_gpt2(dic, styles)

time: 47.8 ms


In [48]:
model.fast_export('../../to_gpt2/recipe54k_0926/',overwrite = True)

make dir
make dir
make dir
make dir
time: 10min 5s
